<a href="https://colab.research.google.com/github/aaltmets/Masterthesis/blob/main/Copy_of_data_pulling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install tweepy==4.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 KB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.13.0
    Uninstalling tweepy-4.13.0:
      Successfully uninstalled tweepy-4.13.0


In [ ]:
import tweepy
import pandas as pd
import configparser
import time


In [ ]:
api_key = 'your key'
api_key_secret = 'your key'

access_token = 'your token'
access_token_secret = 'your token'

bearer_token = 'your token '

In [ ]:
#client = tweepy.Client(consumer_key=api_key, 
                                #consumer_secret=api_key_secret,
                                #access_token=access_token, 
                                #access_token_secret=access_token_secret)

client = tweepy.Client(bearer_token = bearer_token)

In [ ]:
#previous query was covid travel, new coronavirus travel, then safe to travel, travel insurance and travel restriction for pre covid we also use virus travel 
#pre covid also has travel anxiety and travel again. Loop pulls tweets only for 15 mins then stops therefore nr of words is limited 

query = 'COVID travel -is:retweet lang:en has:geo' 

tweets = []
for tweet in tweepy.Paginator(client.search_all_tweets, 
                                 query = query,
                                 user_fields = ['username', 'name', 'public_metrics', 'description', 'location', 'verified', 'created_at', 'entities', 'profile_image_url'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text', 'source', 'context_annotations'],
                                 place_fields = ['geo','id','country', 'country_code'],
                                 expansions = ['geo.place_id', 'author_id'],
                                 start_time = '2019-01-01T00:00:00Z',
                                 end_time = '2020-09-24T00:00:00Z',
                              max_results=100):
    time.sleep(1)
    tweets.append(tweet)
  


In [ ]:
result = []
user_dict = {}



for tweet in tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in tweet.includes['users']:
        user_dict[user.id] = {'username': user.username,
                              'name': user.name, 
                              'followers': user.public_metrics['followers_count'],
                              'following':user.public_metrics['following_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'verified':user.verified,
                              'profile_image_url': user.profile_image_url,
                              'entities': user.entities,
                              'created_at': user.created_at,
                              'location': user.location
                             }


    for i in tweet.data:
        # For each tweet, find the author's information
        author_info = user_dict[i.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet. It seems that geo.place_id is inside of dictionary to lets try to get it out and equal to place_dict 
     #place_info = place_dict[i.geo['place_id']
        result.append({'author_id': i.author_id, 
                       'username': author_info['username'],
                       'name': author_info['name'],
                       'followers': author_info['followers'],
                       'following': author_info['following'],
                       'verified': author_info['verified'],
                       'profile_image_url':author_info['profile_image_url'],
                       'entities': author_info['entities'],
                       'author_created_at':author_info['created_at'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'tweet_id': i.id,
                       'text': i.text,
                       'created_at': i.created_at,
                       'geo': i.geo,
                       'source':i.source,
                       'retweets': i.public_metrics['retweet_count'],
                       'replies': i.public_metrics['reply_count'],
                       'likes': i.public_metrics['like_count'],
                       'quote_count': i.public_metrics['quote_count']
                      })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [ ]:
def try_literal_eval(e):
    try:
        return ast.literal_eval(e)
    except ValueError:
        return {'place_id': 0}

df['geo'] = df['geo'].str['place_id']

In [ ]:
#I could not add location in normal way. received error. therefore created this workaround 

location = []

for tweet in tweets: 
  for place in tweet.includes['places']:

    location.append({'geo':place.id,
                                  'country': place.country,
                                  'country_code': place.country_code
                                  })
df_location = pd.DataFrame(location)

In [ ]:
result = pd.merge(df, df_location, how = "outer", on =['geo'])

In [ ]:
#because for some reseaon high number of tweets gets duplicated, remove duplicates 
result = result.drop_duplicates(subset=['tweet_id'])

In [ ]:
result.shape

(14946, 23)

In [ ]:
result.to_csv('file_name.csv')

In [ ]:
result = result.reset_index(drop=True)